In [ ]:
%matplotlib inline
import pynbody
from pynbody.analysis import profile
import matplotlib.pylab as plt
import numpy as np
import astropy.units as u
import os
import bokeh
from bokeh.layouts import gridplot, row, column
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

bokeh.io.output_notebook()

In [ ]:
from halo_functions import halo_Wechsler2002, halo_scaled_radius, halo_scaled_density, RHO_C
from cog import compute_cog
from snap_io import load_moria_sim_and_kicked

In [ ]:
import astropy.units as u
M_h = 1e14 * u.solMass
c = halo_Wechsler2002(M_h)
c

In [ ]:
virial_radius = c * halo_scaled_radius(M_h, c)
virial_radius

In [ ]:
SIM_NUMBER = 71002

In [ ]:
moria_sim, kicked_sim = load_moria_sim_and_kicked(SIM_NUMBER)

In [ ]:
moria_sim[0].properties

In [ ]:
# sim_path = '/home/michele/sim/MySimulations/Moria8Gyr_tidal/sim60002/out/'
# snaplist = snapshot_list(sim_path, include_dir=True)
# sim = pynbody.load(snaplist[-1])
# sim.physical_units()
# print(sim.filename)
# print(sim.properties['time'].in_units('Gyr'))
# sim.properties
# Jtot = np.sqrt(((np.multiply(sim['j'].transpose(), sim['mass']).sum(axis=1))**2).sum()) # calculate angular momentum

In [ ]:
# p_all = profile.Profile(s62, max='250 kpc')
# plt.plot(p_all['rbins'].in_units('kpc'),p_all['vr_disp'].in_units('km s^-1'),'k')
# plt.xlabel('$R$ [kpc]'); plt.ylabel('$\sigma_{r}$')

In [ ]:
recompute = False
# npy_file = 'cog.npy'
if not recompute:
    data = np.load('cog{}.npz'.format(SIM_NUMBER))
    times_moria, cog_moria = data['times'], data['cog']
    data = np.load('cog{}_kicked.npz'.format(SIM_NUMBER))
    times_kicked, cog_kicked = data['times'], data['cog']
else:
    times_moria, cog_moria = compute_cog(moria_sim, cache_file='cog{}.npz'.format(SIM_NUMBER));
    times_kicked, cog_kicked = compute_cog(kicked_sim, cache_file='cog{}_kicked.npz'.format(SIM_NUMBER));

Use trange to equalize the range of the simulation between the two 

In [ ]:
bins_sfr = 1000
trange = (min(np.min(times_moria), np.min(times_kicked)), max(np.max(times_moria), np.max(times_kicked)))
trange = (0, min(np.max(times_moria), np.max(times_kicked)))
bins = np.linspace(*trange, bins_sfr)
trange, times_moria[-1], times_kicked[-1]

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sfr_moria, sfr_moria_bins = pynbody.plot.sfh(moria_sim[-1], bins=bins_sfr, trange=trange, range=trange, label='MoRIA', subplot=ax)
sfr_kicked, sfr_kicked_bins = pynbody.plot.sfh(kicked_sim[-1], bins=bins_sfr, trange=trange, range=trange, label='Kicked', subplot=ax)
plt.xlabel('Time [Gyr]', fontsize='x-large')
plt.ylabel('SFR [M$_\odot$ yr$^{-1}$]', fontsize='x-large');
plt.legend();

In [ ]:
np.testing.assert_array_equal(sfr_moria_bins, sfr_kicked_bins)

In [ ]:
if recompute:
    del moria_sim, kicked_sim
import gc
gc.collect()

In [ ]:
cog_moria.shape, times_moria.shape

In [ ]:
def rebin_cog(sfr_times, sim_times, cog):
    rebin_x_cog = np.interp(sfr_times, sim_times, cog[0,:])
    rebin_y_cog = np.interp(sfr_times, sim_times, cog[1,:])
    return rebin_x_cog, rebin_y_cog

In [ ]:
t_sfr_moria = sfr_moria_bins[:-1]
t_sfr_kicked = sfr_kicked_bins[:-1]

x_cog_moria, y_cog_moria = rebin_cog(t_sfr_moria, times_moria, cog_moria)
x_cog_kicked, y_cog_kicked = rebin_cog(t_sfr_kicked, times_kicked, cog_kicked)

In [ ]:
np.testing.assert_array_equal(t_sfr_moria, t_sfr_kicked)

In [ ]:
x_cog, y_cog = cog_moria[0,:], cog_moria[1,:]

In [ ]:
source_moria = ColumnDataSource(data=dict(t=t_sfr_moria,
                                          sfr=sfr_moria,
                                          x_cog=x_cog_moria,
                                          y_cog=y_cog_moria))
source_moria.column_names

In [ ]:
source_kicked = ColumnDataSource(data=dict(t=t_sfr_kicked,
                                           sfr=sfr_kicked,
                                           x_cog=x_cog_kicked,
                                           y_cog=y_cog_kicked))
# t = source_kicked.data['t']

In [ ]:
source = ColumnDataSource(data=dict(t_m=t_sfr_moria,
                                    sfr_m=sfr_moria,
                                    x_cog_m=x_cog_moria,
                                    y_cog_m=y_cog_moria,
                                    t_k=t_sfr_kicked,
                                    sfr_k=sfr_kicked,
                                    x_cog_k=x_cog_kicked,
                                    y_cog_k=y_cog_kicked))

In [ ]:
from bokeh.models.annotations import Span
from bokeh.models import HoverTool

def compare_cog_sfh(source):
    start = (source.data['x_cog'][0], source.data['y_cog'][0])
    end = (source.data['x_cog'][-1], source.data['y_cog'][-1])

    hover = HoverTool(tooltips=[
#             ("(x,y)", "($x_cog, $y_cog)"),
            ("time", "@t"),
        ],
                      names=["cog"])

    tools = 'lasso_select, box_select, tap, wheel_zoom, pan, box_zoom, reset'
    
    # COG figure
    f1 = figure(width=400, height=400, title='Center of gravity position',
                x_axis_label="x (kpc)", y_axis_label="y (kpc)",
                tools=tools)
    f1.circle('x_cog', 'y_cog', source=source, name='cog')
    f1.circle(*start, color='yellow')
    f1.circle(*end, color='red')
    f1.circle(0,0, radius=virial_radius.value, alpha=0.1)
    f1.x(0,0)
    f1.add_tools(hover) # do not hover on the virial radius!

    # SFH figure
    f2 = figure(width=400, height=400, title='SFH',
                x_axis_label="Time (Gyr)", y_axis_label="SFR (Msol/yr)",
                tools=tools)
    # f2.tools.pop(4)  # remove pan
    
    # f2.line(t[:-1], sfr)
    f2.vbar(x='t', top='sfr', width=.1, source=source)
    
    
#     t = source.data['t']
#     sfr = source.data['sfr']
#     interval = t[1]-t[0]
#     f2.quad(top='sfr', bottom=np.zeros_like(t), left=t,
#             right=t+interval, alpha=0.5)#, color="#B3DE69")
    simulation_start_line = Span(location=times_moria[0], dimension='height', line_color='firebrick', line_width=1)
    f2.add_layout(simulation_start_line)
    p = row([f1, f2])
    # p = gridplot([[f1, f2]])
    show(p)
    return f1, f2

In [ ]:
compare_cog_sfh(source_moria)

In [ ]:
f1, f2 = compare_cog_sfh(source_kicked)

# Profiles

### Circular velocity profiles

In [ ]:
h1 = moria_sim[0]
c = pynbody.analysis.halo.center(h1, retcen=True)
print("Recentering into: kpc", c)
pynbody.analysis.halo.center(h1)
# print(c)
# h1.properties['eps'] = pynbody.units.Unit("1.1 kpc")
my_eps_value = 0.1
# Beware of the dtype of the SimArray (if something goes wrong remember to "del h1['eps']")
h1['eps'] = pynbody.array.SimArray(np.ones(len(h1['pos'])), "kpc", dtype=np.float32) * my_eps_value
print(h1['eps'])
pd = pynbody.analysis.profile.Profile(h1.d, max=50, type = 'log')
ps = pynbody.analysis.profile.Profile(h1.s, max=50, type = 'log')
pg = pynbody.analysis.profile.Profile(h1.g, max=50, type = 'log')
p = pynbody.analysis.profile.Profile(h1, min=.01,max=50, type = 'log')

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
for prof, name in zip([p,pd,ps,pg],['total','dm','stars','gas']) : ax.plot(prof['rbins'], prof['v_circ'], label=name)
plt.xlabel('$R$ [kpc]');
plt.ylabel('$v_{circ}$ [km/s]');
plt.legend()

In [ ]:
h1['mass'].dtype, h1['pos'].dtype

### Velocity dispersion of stars

In [ ]:
# f1 = figure(width=400, height=400, title='Center of gravity position', x_axis_label="x (kpc)", y_axis_label="y (kpc)")
# f1.line(x_cog, y_cog)
# f1.circle(0,0, radius=virial_radius.value, alpha=0.1)
# f2 = figure(width=400, height=400, x_axis_label="Time (Gyr)", y_axis_label="SFR (Msol/yr)")
# # f2.line(t[:-1], sfr)
# f2.vbar(x=t[:-1], top=sfr, width=None)
# p = row([f1, f2])
# show(p)

In [ ]:
# from bokeh.layouts import layout, row, widgetbox
# from bokeh.models import (
#     ColumnDataSource, HoverTool, TextInput, SingleIntervalTicker, Slider, Button, Label,
#     CategoricalColorMapper,
# )

# def animate_update():
#     gyr = slider.value + 1
#     if gyr > years[-1]:
#         gyr = years[0]
#     slider.value = gyr

# def animate():
#     if button.label == '► Play':
#         button.label = '❚❚ Pause'
#         curdoc().add_periodic_callback(animate_update, 200)
#     else:
#         button.label = '► Play'
#         curdoc().remove_periodic_callback(animate_update)

# button = Button(label='► Play', width=60)
# button.on_click(animate)

In [ ]:
import numpy as np
from bokeh.layouts import layout, row, widgetbox
from bokeh.models import (
    ColumnDataSource, HoverTool, TextInput, SingleIntervalTicker, Slider, Button, Label,
    CategoricalColorMapper,
)
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.plotting import figure, show

def modify_doc(doc):

    # Set up data
    N = 200
    x = np.linspace(0, 4*np.pi, N)
    y = np.sin(x)
    source = ColumnDataSource(data=dict(x=x, y=y))


    # Set up plot
    plot = figure(plot_height=400, plot_width=400, title="my sine wave",
                  tools="crosshair,pan,reset,save,wheel_zoom",
                  x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    # Set up widgets
    text = TextInput(title="title", value='my sine wave')
    offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
    amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
    phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
    freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)

    # Set up callbacks
    def update_title(attrname, old, new):
        plot.title.text = text.value

    text.on_change('value', update_title)

    def update_data(attrname, old, new):

        # Get the current slider values
        a = amplitude.value
        b = offset.value
        w = phase.value
        k = freq.value

        # Generate the new curve
        x = np.linspace(0, 4*np.pi, N)
        y = a*np.sin(k*x + w) + b

        source.data = dict(x=x, y=y)

    for w in [offset, amplitude, phase, freq]:
        w.on_change('value', update_data)

    # Set up layouts and add to document
    inputs = widgetbox(text, offset, amplitude, phase, freq)

    doc.add_root(row(inputs, plot, width=800))
    doc.title = "Sliders"

    # Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)


In [ ]:
doc = app.create_document()
# Show the application
# Make sure the URL matches your Jupyter instance
show(app, notebook_url="localhost:8888")